<a href="https://colab.research.google.com/github/Gwennny/tomato_detect/blob/main/Tomato_Leaf_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U ipykernel

Requirement already up-to-date: ipykernel in /usr/local/lib/python3.7/dist-packages (6.0.1)


In [2]:
!pip install -q streamlit

     |████████████████████████████████| 7.8MB 5.4MB/s 
     |████████████████████████████████| 112kB 41.0MB/s 
     |████████████████████████████████| 4.2MB 41.7MB/s 
     |████████████████████████████████| 174kB 50.9MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 


In [3]:
!pip install pillow

In [4]:
!pip install pyngrok

     |████████████████████████████████| 747kB 4.9MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19262 sha256=9aa66872e76cca8cd0e204b2c313000745ac9e3545a24184a9945cf093596c5d
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok


In [5]:
!ngrok authtoken 1urKShmUJK2fQK222udXSBN6D26_3Vr9rHsU8NHwJEYC3yiP9

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%%writefile predict.py
import tensorflow as tf
import numpy as np
import streamlit as st
from PIL import Image
import requests
from io import BytesIO

st.set_option('deprecation.showfileUploaderEncoding', False)
st.title('Tomato Classifier')
st.text('Provide Url Of The Tomato Classification Model')

@st.cache(allow_output_mutation=True)
def load_model():
  model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/model_inception.h5')
  return model

with st.spinner('Loading Model Into Memory......'):
  model = load_model()

classes = ['Bacterial_spot', 'Early_blight', 'Late_blight', 'Leaf_Mold', 'Septoria_leaf_spot', 'Spider_mites Two-spotted_spider_mite', 'Target_Spot', 'Tomato_Yellow_Leaf_Curl_Virus', 'Tomato_mosaic_virus', 'Healthy']
def scale(image):
  image = tf.cast(image, tf.float32)
  image /= 255.0
  return tf.image.resize(image,[224, 224])

def decode_img(image):
  img = tf.image.decode_jpeg(image, channels =3)
  img = scale(img)
  return np.expand_dims(img, axis=0)
path = st.text_input('Enter Image Url to Classify.. ','https://i2.wp.com/savvygardening.com/wp-content/uploads/2018/01/tomato_plant_disease_septoria_leaf_spot.jpg?w=600&ssl=1')
if path is not None:
  content = requests.get(path).content

  st.write('Predicted Class :')
  with st.spinner('classifying.....'):
    label =np.argmax(model.predict(decode_img(content)),axis=1)
    st.write(classes[label[0]])
  st.write(classes[label[0]])
  image = Image.open(BytesIO(content))
  st.image(image, caption='Classify Tomato Image', use_column_width=True)
  

Writing predict.py


In [8]:
!nohup streamlit run --server.port 80 predict.py &


nohup: appending output to 'nohup.out'


In [9]:
from pyngrok import ngrok
url = ngrok.connect(port=8501)
url

<NgrokTunnel: "http://0f324637024e.ngrok.io" -> "http://localhost:80">

In [10]:
!cat nohup.out